In [1]:
import numpy as np
import os
os.environ['KERAS_BACKEND']        = 'tensorflow'
os.environ['CUDA_DEVICE_ORDER']    = 'PCI_BUS_ID'
os.environ['CUDA_VISIBLE_DEVICES'] = '1, 2'

In [2]:
import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config = config)

import keras.backend as K
K.set_image_data_format('channels_last')

Using TensorFlow backend.


In [3]:
import keras
from keras.models import Model
from keras.layers import Conv2D, Activation, Dropout, Flatten, Dense, Input, MaxPooling2D, AveragePooling2D, BatchNormalization
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from keras.layers.merge import Concatenate, add
from keras.utils import multi_gpu_model

In [4]:
class CUSTOM_HISTORY(keras.callbacks.Callback):
    def init(self):
        self.train_loss = []
        self.val_loss = []
        self.train_acc = []
        self.val_acc = []
    def on_epoch_end(self, batch, logs = {}):
        self.train_loss.append(logs.get('loss'))
        self.val_loss.append(logs.get('val_loss'))
        self.train_acc.append(logs.get('acc'))
        self.val_acc.append(logs.get('val_acc'))
CUSTOM_HIST = CUSTOM_HISTORY()
CUSTOM_HIST.init()

Dict_Class = {'Z' : 0,
              'A' : 0,
              'B' : 0,
              'C' : 1,
              'M' : 1,
              'X' : 1}

def STATISTICS(LABELS, RESULTS):
    H = 0
    F = 0
    M = 0
    N = 0

    assert len(LABELS) == (RESULTS.shape[0])
    
    for I in range(len(LABELS)):
        
        LABEL = LABELS[I]
        RESULT = np.argmax(RESULTS[I])
        
        if LABEL == 1 and RESULT == 1 :
            H += 1
        elif LABEL == 1 and RESULT == 0 :
            M += 1
        elif LABEL == 0 and RESULT == 1 :
            F += 1
        elif LABEL == 0 and RESULT == 0 :
            N += 1
        else :
            pass
        
    ACC = (H + N)/(H + M + F + N)
    POD = H/(H + M)
    FAR = F/(H + F)
    CSI = H/(H + M + F)
    HSS = (2 * (H * N - F * M)) / ((H + M) * (M + N) + (H + F) * (F + N))
    TSS = (H * N - F * M)/ ((H + M) * (F + N))
    
    return ACC, POD, CSI, FAR, HSS, TSS

In [5]:
CONV_INIT = RandomNormal(0, 0.02)
GAMMA_INIT = RandomNormal(1., 0.02)

def CONV(f, *a, **k):
    return Conv2D(f, kernel_initializer = CONV_INIT, *a, **k)

def NORM():
    return BatchNormalization(momentum=0.9, axis=-1, epsilon=1.01e-5, gamma_initializer = GAMMA_INIT)

def POOL(*a, **k):
    return MaxPooling2D(*a, **k)

def BLOCK(INPUT, N_FEATURE):

    N_INCEPTION = N_FEATURE//4
            
    L1 = CONV(N_INCEPTION, (1, 1), strides = 1, padding = 'same') (INPUT)
    L1 = NORM() (L1, training = 1)
    L1 = Activation('relu') (L1)
        
    L2 = CONV(N_INCEPTION, (1, 3), strides = 1, padding = 'same') (INPUT)
    L2 = NORM() (L2, training = 1)
    L2 = Activation('relu') (L2)

    L3 = CONV(N_INCEPTION, (3, 1), strides = 1, padding = 'same') (INPUT)
    L3 = NORM() (L3, training = 1)
    L3 = Activation('relu') (L3)

    L4 = CONV(N_INCEPTION, (3, 3), strides = 1, padding = 'same') (INPUT)
    L4 = NORM() (L4, training = 1)
    L4 = Activation('relu') (L4)

    L = Concatenate(-1)([L1, L2, L3, L4])
    L = CONV(N_INCEPTION * 2, (1, 1), strides = 1, padding = 'same') (L)
    L = Activation('relu') (L)
    L = CONV(N_FEATURE,     (3, 3), strides = 1, padding = 'same') (L)
    L = Activation('relu') (L)

    return L

def ESNET(ISIZE, NC_IN, NC_OUT, GROWTH_RATE, MAX_BLOCK) :

    INPUT = Input(shape = (ISIZE, ISIZE, NC_IN))

    N_FEATURE = 64
    L = CONV(N_FEATURE, (3, 3), strides = 2, padding = 'same') (INPUT)
    L = Activation('relu') (L)
    L = POOL(pool_size = (3, 3), strides = 2, padding = 'same') (L)

    for I in range(MAX_BLOCK):

        IP = L
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)
        
        IP = Concatenate(-1)([IP, L])
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)
        
        
        IP = Concatenate(-1)([IP, L])
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)
        
        IP = Concatenate(-1)([IP, L])
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)    
        
        IP = Concatenate(-1)([IP, L])
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)    
        
        IP = Concatenate(-1)([IP, L])
        N_FEATURE += GROWTH_RATE
        L = BLOCK(IP, N_FEATURE)
        
        L = POOL(pool_size = (3, 3), strides = 2, padding = 'same') (L)

    SHAPE = L.get_shape().as_list()

    L = AveragePooling2D(pool_size = (SHAPE[1], SHAPE[2]), strides = (SHAPE[1], SHAPE[2]), padding = 'same') (L)
    
    L = Flatten() (L)
    
#    L = Dense(units = 2048, activation = 'relu') (L)
#    L = Dropout(0.5) (L)

#    L = Dense(units = 2048, activation = 'relu') (L)
#    L = Dropout(0.5) (L)

    L = Dense(units = NC_OUT) (L)
    L = Activation('softmax') (L)

    return Model(inputs = INPUT, outputs = L)

def weighted_categorical_crossentropy(weights):
    weights = K.variable(weights)
    def loss(y_true, y_pred):
        y_pred /= K.sum(y_pred, axis=-1, keepdims=True)
        y_pred = K.clip(y_pred, K.epsilon(), 1 - K.epsilon())
        loss = y_true * K.log(y_pred) * weights
        loss = -K.sum(loss, -1)
        return loss
    return loss

In [6]:
ISIZE = 1024
MAX_BLOCK = 4
NC_IN = 1
NC_OUT = 2
BATCH_SIZE = 4
GROWTH_RATE = 16
MAX_EPOCHS = 100

NAME_MODEL = 'MGNT_M_' + '%04d'%ISIZE + '_MxBk' + str(MAX_BLOCK)
DATASET_NAME = 'ESPARK_OLD_M'

PATH_SAVE = './FlareMagNet_MODEL/' + NAME_MODEL + '/'
os.mkdir(PATH_SAVE) if not os.path.exists(PATH_SAVE) else None

#SUMMARY = open(PATH_SAVE + 'SUMMARY.txt', 'w')
#SUMMARY.writelines('EPOCH    ACC    POD    CSI    FAR    HSS    TSS' + '\n')

with tf.device('/cpu:0'):
    MODEL = ESNET(ISIZE, NC_IN, NC_OUT, GROWTH_RATE, MAX_BLOCK)
    MODEL.summary()

from keras.optimizers import Adam
MULTI_MODEL = multi_gpu_model(MODEL, gpus = 2)
ADAM = Adam(lr = 2e-4)
WEIGHTS = np.array([1.0,1.0])
MULTI_MODEL.compile(loss=weighted_categorical_crossentropy(WEIGHTS), optimizer=ADAM, metrics=['accuracy'])

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1024, 1024, 1 0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 512, 512, 64) 640         input_1[0][0]                    
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 512, 512, 64) 0           conv2d_1[0][0]                   
__________________________________________________________________________________________________
max_pooling2d_1 (MaxPooling2D)  (None, 256, 256, 64) 0           activation_1[0][0]               
__________________________________________________________________________________________________
conv2d_2 (

In [7]:
from keras.preprocessing.image import ImageDataGenerator

PATH_TRAIN = './FlareMagNet_DATA/' + DATASET_NAME + '/TRAIN/'
PATH_TEST = './FlareMagNet_DATA/' + DATASET_NAME + '/TEST/'
COLOR_MODE = 'grayscale'

TRAIN_GEN = ImageDataGenerator(rescale = 1./255.)#, validation_split = 0.1)
TRAIN_GENERATOR = TRAIN_GEN.flow_from_directory(PATH_TRAIN, target_size=(ISIZE, ISIZE),
                                          batch_size=BATCH_SIZE, class_mode='categorical',
                                          color_mode = COLOR_MODE)#, subset = 'training')
N_BATCH_TRAIN = int(TRAIN_GENERATOR.n / BATCH_SIZE)

#VAL_GENERATOR = TRAIN_GEN.flow_from_directory(PATH_TRAIN, target_size=(ISIZE, ISIZE),
#                                          batch_size=BATCH_SIZE, class_mode='categorical',
#                                          color_mode = COLOR_MODE, subset = 'validation')
#N_BATCH_VAL = int(VAL_GENERATOR.n / BATCH_SIZE)


TEST_GEN = ImageDataGenerator(rescale = 1./255.)
TEST_GENERATOR = TEST_GEN.flow_from_directory(PATH_TEST, target_size=(ISIZE, ISIZE),
                                         batch_size=BATCH_SIZE, class_mode='categorical',
                                         color_mode = COLOR_MODE, shuffle = False)
N_BATCH_TEST = int(TEST_GENERATOR.n / BATCH_SIZE)

Found 4268 images belonging to 2 classes.
Found 2972 images belonging to 2 classes.


In [8]:
EPOCH = 1

while EPOCH <= MAX_EPOCHS :

    MULTI_MODEL.fit_generator(TRAIN_GENERATOR,
                              steps_per_epoch = N_BATCH_TRAIN, 
                              validation_data = TEST_GENERATOR,
                              validation_steps = N_BATCH_TEST,
                              epochs = 1,
                              callbacks = [CUSTOM_HIST])

    CLASSES = TEST_GENERATOR.classes
    RESULT = MULTI_MODEL.predict_generator(TEST_GENERATOR)
    
    ACC, POD, CSI, FAR, HSS, TSS = STATISTICS(CLASSES, RESULT)
    LINE = '%5d'%EPOCH + '%7.4f'%ACC + '%7.4f'%POD + '%7.4f'%CSI + '%7.4f'%FAR + '%7.4f'%HSS + '%7.4f'%TSS
    print(LINE)
    #SUMMARY.writelines(LINE + '\n')
    
    #NAME_SAVE = PATH_SAVE + NAME_MODEL + '_' + '%04d'%EPOCH + '.h5'
    #MODEL.save(NAME_SAVE)
    
    EPOCH += 1

#SUMMARY.close()

Epoch 1/1
1067/1067 [==============================] - 956s 896ms/step - loss: 0.4678 - acc: 0.8065 - val_loss: 0.4309 - val_acc: 0.8580


ZeroDivisionError: division by zero

In [ ]:
LALA = open(PATH_SAVE + 'LALA.txt', 'w')

LEN = len(CUSTOM_HIST.train_loss)

for I in range(LEN):
    
    EP = I + 1
    TL = CUSTOM_HIST.train_loss[I]
    TA = CUSTOM_HIST.train_acc[I]
    VL = CUSTOM_HIST.val_loss[I]
    VA = CUSTOM_HIST. val_acc[I]
    
    LA = '%4d %7.5f %7.5f %7.5f %7.5f' % (EP, TL, TA, VL, VA)
    print(LA)
    LALA.writelines(LA + '\n')

LALA.close()

In [ ]:
import matplotlib.pyplot as plt

FIG, LOSS_AX = plt.subplots()
ACC_AX = LOSS_AX.twinx()
LOSS_AX.plot(CUSTOM_HIST.train_loss, 'y', label = 'TRAIN LOSS')
LOSS_AX.plot(CUSTOM_HIST.val_loss, 'r', label = 'VAL LOSS')
ACC_AX.plot(CUSTOM_HIST.train_acc, 'b', label = 'TRAIN ACC')
ACC_AX.plot(CUSTOM_HIST.val_acc, 'g', label = 'VAL ACC')
LOSS_AX.set_xlabel('EPOCH')
LOSS_AX.set_ylabel('LOSS')
ACC_AX.set_ylabel('ACCURACY')
LOSS_AX.legend(loc='upper left')
ACC_AX.legend(loc='lower left')
FIG.savefig('./' + NAME_MODEL + '.png')